# <center>Welcome FSP solved with diverse heuristics</center>

## Data Utils

In [1]:
import numpy as np
import itertools
import time
import math
import pandas as pd
import matplotlib.pyplot as plt

In [2]:


'''
Reading the data from the benchmark file
'''

def read_flow_shop_data(file_path, machine_count, job_count):
    instances = []
    with open(file_path) as p:
        lines = p.readlines()
        line_count = len(lines)

        instance_count = line_count // (machine_count + 3)

        for i in range(instance_count):
            # recover the data of each instance
            params_line = lines[i * (machine_count + 3) + 1]
            job_count, machine_count, initial_seed, upper_bound, lower_bound = list(
                map(lambda x: int(x), params_line.split()))

            # processing_times = [list(map(int, lines[i * (machine_count + 3) + 3])) for line in lines]
            processing_times = np.array([list(map(lambda x: int(x), line.strip().split())) for
                                         line in lines[
                                                 i * (machine_count + 3) + 3:  # start
                                                 i * (machine_count + 3) + 3 + machine_count  # end
                                                 ]
                                         ])

            record = (machine_count, job_count, processing_times)
            instances.append(record)

    return instances



### Path Cost calculation function :
used to calculate the cost of current node, which is the correct cost starting for the actual path of executed jobs

In [3]:
def incremental_cost(machine_job_matrix, jobs_sequence):
    nb_jobs = len(jobs_sequence)
    nb_machines = machine_job_matrix.shape[0]

    incremental_cost = np.zeros((nb_machines, nb_jobs))

    # evaluate the first machines
    incremental_cost[0, 0] = machine_job_matrix[0][jobs_sequence[0]]

    for i in range(1, nb_jobs):
        incremental_cost[0][i] = incremental_cost[0][i - 1] + \
            machine_job_matrix[0][jobs_sequence[i]]

    # evaluate the rest of machines
    for i in range(1, nb_machines):
        incremental_cost[i, 0] = incremental_cost[i - 1, 0] + \
            machine_job_matrix[i, jobs_sequence[0]]
        for j in range(1, nb_jobs):
            incremental_cost[i, j] = machine_job_matrix[i, jobs_sequence[j]] + \
                max(incremental_cost[i - 1, j], incremental_cost[i, j - 1])
    return incremental_cost[-1,-1]

### Gantt graph generator

In [4]:
def generate_gantt_chart(current_instance, solution):
    plt.figure(figsize=(20, 12))
    df = pd.DataFrame(columns=['Machine', 'Job', 'Start', 'Finish'])

    machines, jobs = current_instance.shape
    machine_times = np.zeros((machines, jobs))
    start_time_m = np.zeros(machines)
    for job in solution:

        for machine_index in range(machines):
            start_time = start_time_m[machine_index]
            if machine_index > 0:
                start_time = max(start_time, start_time_m[machine_index-1])
            end_time = start_time + \
                current_instance[machine_index, job]
            start_time_m[machine_index] = end_time

            df = pd.concat([df, pd.DataFrame({'Machine': f'Machine {machine_index + 1}',
                                                'Job': f'Job {job + 1}',
                                                'Start': start_time,
                                                'Finish': end_time}, index=[0])], ignore_index=True)

            machine_times[machine_index, job] = end_time

    colors = plt.cm.tab10.colors
    for i, machine_index in enumerate(range(machines)):
        machine_df = df[df['Machine'] == f'Machine {machine_index + 1}']
        plt.broken_barh([(start, end - start) for start, end in zip(machine_df['Start'], machine_df['Finish'])],
                        (i * 10, 9), facecolors=[colors[j % 10] for j in range(jobs)], edgecolor='black')

    plt.xlabel('Time')
    plt.yticks([i * 10 + 4.5 for i in range(machines)],
                [f'Machine {i + 1}' for i in range(machines)])
    plt.show()

## Heuristics

### Johnson **n** jobs **2** machines

In [5]:
def johnson_method(processing_times):
    jobs, machines = processing_times.shape
    copy_processing_times = processing_times.copy()
    maximum = processing_times.max() + 1
    m1 = []
    m2 = []
    
    if machines != 2:
        raise Exception("Johson method only works with two machines")
        
    for i in range(jobs):
        minimum = copy_processing_times.min()
        position = np.where(copy_processing_times == minimum)
        
        if position[1][0] == 0:
            m1.append(position[0][0])
        else:
            m2.insert(0, position[0][0])
        
        copy_processing_times[position[0][0]] = maximum
        
    return m1+m2

### Test Johnson

In [6]:
# Generate a random example to work with 7 jobs and 2 machines
rnd_data = np.random.randint(size=(7,2), low=5, high=23)
print(rnd_data, "\n")

start_time = time.time()
sol = johnson_method(rnd_data)
elapsed_time = time.time() - start_time

print(f'Best sequence found by Johnson is {sol} with a makespan of {incremental_cost(np.array(rnd_data).T,sol)}')
print("Elapsed time:", elapsed_time, "seconds")

[[19  9]
 [21 10]
 [ 5  8]
 [10 22]
 [ 7  5]
 [16  5]
 [ 5  9]] 

Best sequence found by Johnson is [2, 6, 3, 1, 0, 5, 4] with a makespan of 88.0
Elapsed time: 0.0 seconds


### CDS Heuristic [Campbell,Dudek and Smith ] : **n** jobs **m** machines (1970)

In [7]:
# python code for CDS heuristic

def cds_heuristic(processing_times):
    
    
    nb_machines, nb_jobs = processing_times.shape


    best_cost = math.inf

    
    machine_1_times = np.zeros((nb_jobs,1))
    machine_2_times = np.zeros((nb_jobs,1))
    
    
    # iterate through the nb_machines-1 auxiliary n-job 2-machines problems



    for k in range(nb_machines -1):
        machine_1_times[:,0] += processing_times[:][k]
        machine_2_times[:,0] += processing_times[:][-k-1]
        
        jn_times = np.concatenate((machine_1_times, machine_2_times), axis=1)
        seq = johnson_method(jn_times)
        cost = incremental_cost(jn_times.T,seq)
        if cost < best_cost:
            best_cost = cost
            best_seq = seq
    
    return best_seq, best_cost


### Test CDS

In [8]:
rnd_data = np.random.randint(size=(8,5), low=10, high=50)
print(rnd_data, "\n")
cds_heuristic(rnd_data.T)
    


[[14 23 31 31 39]
 [28 44 38 13 42]
 [20 18 13 22 22]
 [26 49 12 28 39]
 [39 12 36 44 15]
 [25 35 15 27 22]
 [22 33 40 24 28]
 [45 29 11 24 19]] 



([0, 2, 6, 3, 1, 5, 7, 4], 240.0)

In [9]:
instances = read_flow_shop_data('../Lab1/data/tai20_5.txt', 5, 20)
comman_instance = instances[0][2]

In [10]:
start_time = time.time()
best_solution,best_cost  = cds_heuristic(comman_instance)


best_cost = incremental_cost(comman_instance,best_solution)

end_time = time.time()

elapsed_time = end_time - start_time

print(f'Results of CDS:')
print(f'Best sequence is {best_solution} with a makespan of {best_cost}.')
print(f'Elapsed time of {elapsed_time} seconds.')



Results of CDS:
Best sequence is [14, 2, 8, 13, 16, 7, 6, 0, 18, 3, 10, 15, 11, 1, 4, 5, 19, 17, 9, 12] with a makespan of 1390.0.
Elapsed time of 0.0 seconds.


### NEH Algorithm

In [11]:
def order_jobs_in_descending_order_of_total_completion_time(processing_times):
    total_completion_time = processing_times.sum(axis=1)
    return np.argsort(total_completion_time, axis=0).tolist()

In [12]:
def insertion(sequence, position, value):
    new_seq = sequence[:]
    new_seq.insert(position, value)
    return new_seq

In [13]:
def neh_algorithm(processing_times):
    ordered_sequence = order_jobs_in_descending_order_of_total_completion_time(processing_times)
    # Define the initial order
    J1, J2 = ordered_sequence[:2]
    sequence = [J1, J2] if incremental_cost(processing_times.T,[J1, J2]) < incremental_cost(processing_times.T,[J2, J1]) else [J2, J1]
    del ordered_sequence[:2]
    # Add remaining jobs
    for job in ordered_sequence:
        Cmax = float('inf')
        best_sequence = []
        for i in range(len(sequence)+1):
            new_sequence = insertion(sequence, i, job)
            Cmax_eval = incremental_cost(processing_times.T,new_sequence)
            if Cmax_eval < Cmax:
                Cmax = Cmax_eval
                best_sequence = new_sequence
        sequence = best_sequence
    return sequence, Cmax

In [14]:
start_time = time.time()
best_solution,best_cost  = neh_algorithm(instances[6][2].T)

end_time = time.time()

elapsed_time = end_time - start_time

print(f'Results of CDS:')
print(f'Best sequence is {best_solution} with a makespan of {best_cost}.')
print(f'Elapsed time of {elapsed_time} seconds.')


Results of CDS:
Best sequence is [4, 2, 19, 10, 7, 5, 3, 8, 1, 12, 6, 18, 16, 9, 14, 15, 0, 17, 13, 11] with a makespan of 1284.0.
Elapsed time of 0.015420675277709961 seconds.


### HAM Heuristic

In [15]:
from utils.benchmarks import benchmarks

In [16]:
def incremental_cost(processing_times, sequence):
    _, num_machines = processing_times.shape
    num_jobs = len(sequence)
    completion_times = np.zeros((num_jobs, num_machines))
    
    # Calculate the completion times for the first machine
    completion_times[0][0] = processing_times[sequence[0]][0]
    for i in range(1, num_jobs):
        completion_times[i][0] = completion_times[i-1][0] + processing_times[sequence[i]][0]
    
    # Calculate the completion times for the remaining machines
    for j in range(1, num_machines):
        completion_times[0][j] = completion_times[0][j-1] + processing_times[sequence[0]][j]
        for i in range(1, num_jobs):
            completion_times[i][j] = max(completion_times[i-1][j], completion_times[i][j-1]) + processing_times[sequence[i]][j]
    
    # Return the total completion time, which is the completion time of the last job in the last machine
    return completion_times[num_jobs-1][num_machines-1]

In [34]:
def ham_sol_1(Pi1: np.ndarray, Pi2: np.ndarray) -> list:
    diff = Pi2 - Pi1
    sol = np.argsort(diff, axis=0).tolist()
    sol.reverse()       # in decreasing order
    return sol

In [40]:
def ham_sol_2(Pi1: np.ndarray, Pi2: np.ndarray) -> list:
    diff = Pi2 - Pi1
    Pi1_index = [(x,i) for i,x in enumerate(Pi1) if diff[i]>=0]
    Pi2_index = [(x,i) for i,x in enumerate(Pi2) if diff[i]< 0]
    
    Pi1_sorted = sorted(Pi1_index)
    Pi2_sorted = sorted(Pi2_index, reverse=True)

    Pi1_list = [x[1] for x in Pi1_sorted]
    Pi2_list = [x[1] for x in Pi2_sorted]

    return Pi1_list + Pi2_list

In [41]:
def ham_heuristic(processing_times: np.ndarray) -> list:
    _ , m = processing_times.shape
    left = processing_times[:, :int(m/2)]
    right = processing_times[:, int(m/2):]

    Pi1 = left.sum(axis=1)
    Pi2 = right.sum(axis=1)

    solution1 = ham_sol_1(Pi1, Pi2)
    solution2 = ham_sol_2(Pi1, Pi2)
    Cmax1 = incremental_cost(processing_times, solution1)
    Cmax2 = incremental_cost(processing_times, solution2)
    
    if Cmax1 < Cmax2:
        return solution1, Cmax1
    else:
        return solution2, Cmax2

### TEST HAM

In [48]:
start_time = time.time()
best_solution, best_cost  = ham_heuristic(benchmarks[6])

end_time = time.time()

elapsed_time = end_time - start_time

print(f'Results of HAM:')
print(f'Best sequence is {best_solution} with a makespan of {best_cost} and size {len(best_solution)}.')
print(f'Elapsed time of {elapsed_time} seconds.')
print("Sa7a ftrouk chakib , macht ? hmdlh, oui oui mchat XD ")
print("Allah yselmek bro, saha ftorrek ou ya3tik sa7a ")

Results of HAM:
Best sequence is [4, 10, 7, 14, 2, 1, 12, 19, 5, 0, 15, 9, 8, 16, 6, 17, 18, 11, 13, 3] with a makespan of 1334.0 and size 20.
Elapsed time of 0.0 seconds.
Sa7a ftrouk chakib , macht ? hmdlh, oui oui mchat XD 
Allah yselmek bro, saha ftorrek ou ya3tik sa7a 
